In [13]:
import re
from collections import Counter
import itertools
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
from nltk.corpus import movie_reviews
import random

# Thay đổi vector biểu diễn dưới dạng

```
{'contains(waste)':False, 'contains(lot)': False, ... }
```

```
{'waste':10, 'lot': 5, ...} Từ - Tấn số
```

In [14]:
documents = [
    (list(movie_reviews.words(fileid)), category)
    for category in movie_reviews.categories()
    for fileid in movie_reviews.fileids(category)
]
random.shuffle(documents)

In [20]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words()) 

word_features = list(all_words)[:2000]

# Chuyển sang vector tần số: {'word': count}
def document_features_binary(document):  
    document_words = set(document)  
    features = {} 
    for word in word_features: 
        features['contains({})'.format(word)] = (word in document_words) 
    return features

In [21]:
print(document_features_binary(movie_reviews.words('pos/cv957_8737.txt')))

{'contains(,)': True, 'contains(the)': True, 'contains(.)': True, 'contains(a)': True, 'contains(and)': True, 'contains(of)': True, 'contains(to)': True, "contains(')": True, 'contains(is)': True, 'contains(in)': True, 'contains(s)': True, 'contains(")': True, 'contains(it)': True, 'contains(that)': True, 'contains(-)': True, 'contains())': True, 'contains(()': True, 'contains(as)': True, 'contains(with)': True, 'contains(for)': True, 'contains(his)': True, 'contains(this)': True, 'contains(film)': False, 'contains(i)': False, 'contains(he)': True, 'contains(but)': True, 'contains(on)': True, 'contains(are)': True, 'contains(t)': False, 'contains(by)': True, 'contains(be)': True, 'contains(one)': True, 'contains(movie)': True, 'contains(an)': True, 'contains(who)': True, 'contains(not)': True, 'contains(you)': True, 'contains(from)': True, 'contains(at)': False, 'contains(was)': False, 'contains(have)': True, 'contains(they)': True, 'contains(has)': True, 'contains(her)': False, 'conta

In [22]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words()) 

word_features = list(all_words)[:2000]

def document_features_counter(document):
    counts = Counter(w.lower() for w in document)
    features = {word: counts.get(word, 0) for word in word_features}
    return features

In [19]:
print(document_features_counter(movie_reviews.words('pos/cv957_8737.txt')))

{',': 33, 'the': 22, '.': 25, 'a': 25, 'and': 10, 'of': 16, 'to': 18, "'": 12, 'is': 9, 'in': 7, 's': 10, '"': 2, 'it': 8, 'that': 4, '-': 9, ')': 5, '(': 5, 'as': 3, 'with': 7, 'for': 6, 'his': 6, 'this': 6, 'film': 0, 'i': 0, 'he': 4, 'but': 3, 'on': 5, 'are': 1, 't': 0, 'by': 1, 'be': 2, 'one': 4, 'movie': 3, 'an': 3, 'who': 1, 'not': 2, 'you': 2, 'from': 4, 'at': 0, 'was': 0, 'have': 1, 'they': 1, 'has': 4, 'her': 0, 'all': 4, '?': 0, 'there': 4, 'like': 1, 'so': 0, 'out': 3, 'about': 2, 'up': 0, 'more': 0, 'what': 1, 'when': 1, 'which': 4, 'or': 0, 'she': 4, 'their': 0, ':': 4, 'some': 0, 'just': 1, 'can': 0, 'if': 1, 'we': 0, 'him': 4, 'into': 2, 'even': 0, 'only': 1, 'than': 0, 'no': 0, 'good': 0, 'time': 0, 'most': 1, 'its': 0, 'will': 1, 'story': 0, 'would': 0, 'been': 0, 'much': 0, 'character': 0, 'also': 4, 'get': 1, 'other': 2, 'do': 1, 'two': 2, 'well': 1, 'them': 2, 'very': 1, 'characters': 0, ';': 0, 'first': 0, '--': 5, 'after': 0, 'see': 0, '!': 1, 'way': 4, 'because':

# Xây dựng hạng biểu diễn encode dạng vector trong sklearn

```
Vocab{waste, lot, many} --> (?, ?, ?)
```

In [23]:
# Hàm chuyển document về chuỗi (nếu đầu vào là tokens)
def doc_to_text(doc):
    words = doc_words_from_document(doc)
    return " ".join(words)

def doc_words_from_document(doc):
    """
    Trích danh sách từ (lowercase) từ một document. doc có thể là (list_tu, nhãn), hoặc list_tu, hoặc string.
    """
    # Nếu là tuple (words, label)
    if isinstance(doc, tuple) and len(doc) >= 1:
        doc = doc[0]
    if isinstance(doc, list):
        words = [str(w).lower() for w in doc]
    elif isinstance(doc, str):
        # Tokenize đơn giản cho tiếng Việt/Anh (giữ chữ/số)
        words = re.findall(r"\w+", doc.lower())
    else:
        words = []
    return words

In [24]:
# 2Encode dạng vector trong sklearn (CountVectorizer)
# Xây dựng corpus văn bản
if isinstance(documents, list):
    corpus_texts = [doc_to_text(d) for d in documents[:200]]  # dùng 200 tài liệu đầu
elif isinstance(documents, str):
    corpus_texts = [doc_to_text(documents)]


# Xây dựng vocabulary
# Lấy vocab từ chính corpus
tokens_all = []
for t in corpus_texts:
    tokens_all.extend(re.findall(r"\w+", t.lower()))
vocab_for_vectorizer = sorted(set(tokens_all))

vectorizer = CountVectorizer(vocabulary=vocab_for_vectorizer, lowercase=True, token_pattern=r"(?u)\b\w+\b")
X = vectorizer.transform(corpus_texts)

In [25]:
print("Kích thước vocab:", len(vectorizer.vocabulary_))
print("Kích thước ma trận (số văn bản, số đặc trưng):", X.shape)

Kích thước vocab: 13981
Kích thước ma trận (số văn bản, số đặc trưng): (200, 13981)


In [26]:
# Xem vector của văn bản đầu tiên
first_vec = X[0].toarray()[0]

# Lấy các đặc trưng khác 0 để quan sát
inv_vocab = {idx: term for term, idx in vectorizer.vocabulary_.items()}
non_zero = [(inv_vocab[i], int(c)) for i, c in enumerate(first_vec) if c > 0]
print("Đặc trưng khác 0 của văn bản [0]:", non_zero[:20])

Đặc trưng khác 0 của văn bản [0]: [('1', 2), ('1989', 1), ('2', 4), ('20', 1), ('5', 1), ('9', 1), ('a', 50), ('about', 7), ('absolutely', 1), ('abuses', 2), ('accuracy', 5), ('accurate', 2), ('achieved', 1), ('across', 1), ('acting', 1), ('actor', 1), ('acts', 1), ('actually', 1), ('actualy', 1), ('ad', 1)]
